In [ ]:
import pyzed.sl as sl
import math
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt 
%matplotlib inline
import os
def main():
    
    path = 'C:/Users/Akash Jain/Documents/ZED/Videos/'
    video_list  = os.listdir(path)
    for video_name in video_list:
        filepath = path+video_name
        print(filepath)
        init = sl.InitParameters(svo_input_filename=filepath,svo_real_time_mode=False)
        zed = sl.Camera()

        # Create a InitParameters object and set configuration parameters
        err = zed.open(init)
        if err != sl.ERROR_CODE.SUCCESS:
            exit(1)

        # Create and set RuntimeParameters after opening the camera
        runtime_parameters = sl.RuntimeParameters()
        runtime_parameters.sensing_mode = sl.SENSING_MODE.SENSING_MODE_STANDARD  # Use STANDARD sensing mode

        
        i = 0
        image = sl.Mat()
        depth = sl.Mat()
        point_cloud = sl.Mat()
        frame_count = zed.get_svo_number_of_frames()
        print("frame_count",frame_count)
        while i < frame_count:
            itr=0
            # A new image is available if grab() returns SUCCESS            
            for itr in range(10):                
                if zed.grab(runtime_parameters) == sl.ERROR_CODE.SUCCESS:                    
                    zed.retrieve_image(image, sl.VIEW.VIEW_LEFT)
                itr= itr+1
                sys.stdout.flush()
            if zed.grab(runtime_parameters) == sl.ERROR_CODE.SUCCESS:
                zed.retrieve_image(image, sl.VIEW.VIEW_LEFT)
                image_size = zed.get_resolution()
                width = image_size.width
                height = image_size.height
                # Retrieve depth map. Depth is aligned on the left image
                zed.retrieve_measure(depth, sl.MEASURE.MEASURE_DEPTH)
                # Retrieve colored point cloud. Point cloud is aligned on the left image.
                zed.retrieve_measure(point_cloud, sl.MEASURE.MEASURE_XYZRGBA)
                # We measure the distance camera - object using Euclidean distance
                image_array=[]
                image_array = np.zeros([width,height])
                for wd in range(width):
                    for ht in range(height):
                        err, point_cloud_value = point_cloud.get_value(wd, ht)

                        distance = math.sqrt(point_cloud_value[0] * point_cloud_value[0] +
                                             point_cloud_value[1] * point_cloud_value[1] +
                                             point_cloud_value[2] * point_cloud_value[2])

                        if not np.isnan(distance) and not np.isinf(distance):
                            distance = round(distance/1000,2)
                            image_array[wd,ht] = distance
                        else:
                            image_array[wd,ht] = 0
                sys.stdout.flush()
                i = i+10
                np.savetxt(video_name+"_"+str(i)+"_.csv",image_array,delimiter=',')
                filepath1= 'C:/Users/Akash Jain/Documents/ZED/Images/'
                image.write(filepath1+video_name+"_"+str(i)+".jpg")
            else:
                break
        # Close the camera
        zed.close()
if __name__ == "__main__":
    main()